<a href="https://colab.research.google.com/github/ThiagoMueller/csci2470labs/blob/main/LIME_Lab_Appendix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: LIME Appendix

This is a quick demonstration on how to save the Keras model in a JSON file and how to save and load trained weights.

In [ ]:
import os
import sys

isColab = "google.colab" in sys.modules
# this also works:
# isColab = "COLAB_GPU" in os.environ

if isColab:
    # os.environ["SDL_VIDEODRIVER"] = "dummy"
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)

    colab_path = ("/content/drive/Shared drives"
        + "/CS1470 TAs Fall 2022/Labs/lab04_lime/released_lab")
    sys.path.append(colab_path)

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
# # Killing optional CPU driver warnings
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import matplotlib.pyplot as plt
import matplotlib as mpl
from preprocess import *

import json

# ensures that we run only on cpu
# this environment variable is not permanent
# it is valid only for this session
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

ModuleNotFoundError: No module named 'preprocess'

In [ ]:
data_path = "data"
model_path = "model"

# If you are working on Colab, you need to modify the path your other files too.
if isColab:
    data_path = f"{colab_path}/{data_path}"
    model_path = f"{colab_path}/{model_path}"

## Data

In [ ]:
# cifar_class_list = ["airplane", "automobile", "bird", "cat", "deer",
#                     "dog", "frog", "horse", "ship", "truck"]

# Read entire CIFAR data
image_train_full, label_train_full = unpickle_CIFAR(f"{data_path}/train")
image_test_full, label_test_full = unpickle_CIFAR(f"{data_path}/test")

# Keep cats and dogs only and throw away the other classes
cifar_class_list = ["cat", "dog"]
image_train_uint, label_train = get_subset(image_train_full, label_train_full,
                                           class_list=cifar_class_list,
                                           num=None)
image_test_uint, label_test = get_subset(image_test_full, label_test_full,
                                         class_list=cifar_class_list,
                                         num=None)

# Shuffle
seed = 42
image_train_uint, label_train = shuffle_data(image_train_uint, label_train, seed)
image_test_uint,  label_test  = shuffle_data(image_test_uint,  label_test,  seed)

# Normalize the inputs and one-hot encode the outputs
image_train = np.float32(image_train_uint/255.0)
image_test  = np.float32(image_test_uint/255.0)
oh_label_train = one_hot_encode(label_train, cifar_class_list)
oh_label_test  = one_hot_encode(label_test,  cifar_class_list)

## Model Import

**This is how you can simply import a Keras model from a JSON file.**

In [ ]:
with open(f"{model_path}/cnn_model_export.json", "r") as readfile:
    cnn_model_export = json.load(readfile)

cnn_model = tf.keras.models.model_from_json(cnn_model_export)
cnn_model_untrained = tf.keras.models.model_from_json(cnn_model_export)
cnn_model.summary()

Model: "cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 16, 16, 16)        1216      
                                                                 
 Conv1-Norm (BatchNormalizat  (None, 16, 16, 16)       512       
 ion)                                                            
                                                                 
 Conv1-LeakyReLU (LeakyReLU)  (None, 16, 16, 16)       0         
                                                                 
 Conv1-Pool (MaxPooling2D)   (None, 16, 16, 16)        0         
                                                                 
 Conv2 (Conv2D)              (None, 8, 8, 20)          8020      
                                                                 
 Conv2-Norm (BatchNormalizat  (None, 8, 8, 20)         128       
 ion)                                                    

**Here is how the model was defined in the first place.**

In [ ]:
# cnn_model = tf.keras.Sequential(
#     layers = [
#         # First Convolution Layer
#         tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(2, 2), padding="same", name="Conv1"),
#         tf.keras.layers.BatchNormalization(axis=[1, 2], momentum=0, center=False, scale=False, name="Conv1-Norm"),
#         tf.keras.layers.LeakyReLU(name="Conv1-LeakyReLU"),
#         tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1), padding="same", name="Conv1-Pool"),
#         # Second Convolution Layer
#         tf.keras.layers.Conv2D(filters=20, kernel_size=5, strides=(2, 2), padding="same", name="Conv2"),
#         tf.keras.layers.BatchNormalization(axis=[1, 2], momentum=0, center=False, scale=False, name="Conv2-Norm"),
#         tf.keras.layers.LeakyReLU(name="Conv2-LeakyReLU"),
#         tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding="same", name="Conv2-Pool"),
#         # Third Convolution Layer
#         tf.keras.layers.Conv2D(filters=20, kernel_size=3, strides=(1, 1), padding="same", name="Conv3"),
#         tf.keras.layers.BatchNormalization(axis=[1, 2], momentum=0, center=False, scale=False, name="Conv3-Norm"),
#         tf.keras.layers.LeakyReLU(name="Conv3-LeakyReLU"),
#         # Three Dense Layers
#         tf.keras.layers.Flatten(name="Flatten"),
#         tf.keras.layers.Dense(160, activation="leaky_relu", name="Dense1"),
#         tf.keras.layers.Dropout(rate=0.3, name="Dropout1"),
#         tf.keras.layers.Dense(20, activation="leaky_relu", name="Dense2"),
#         tf.keras.layers.Dropout(rate=0.3, name="Dropout2"),
#         tf.keras.layers.Dense(2, activation="softmax", name="Dense3"),
#     ],
#     name = "cnn_model"
# )

# cnn_model.build(tf.TensorShape([None, 32, 32, 3]))

## Weight Load

**Here is how to import pretrained weights. In this way, you don't have to train the weights again.**

In [ ]:
cnn_model.load_weights(f"{model_path}/cnn_model_weights_TA")

**This is how the model was trained in the first place.**

In [ ]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
# metrics = [tf.keras.metrics.CategoricalAccuracy()]

# cnn_model.compile(optimizer=optimizer,
#                   loss=loss,
#                   metrics=metrics)

# cnn_model.fit(x = image_train, y = oh_label_train,
#               epochs = 30, batch_size = 64,
#               validation_data = (image_test, oh_label_test), validation_freq = 5)

**Finally, this is how to export a model into a JSON file and save the weights**

In [ ]:
# cnn_model_export = cnn_model.to_json()
# with open(f"{model_path}/cnn_model_export.json", "w", encoding = "utf-8") as outfile:
#     json.dump(cnn_model_export, outfile)

# # you can change the name of the exported weights
# cnn_model.save_weights(f"{model_path}/cnn_model_weights_TA")

In [ ]:
cifar_class_array = np.array(cifar_class_list)
predicted_labels = cifar_class_array[tf.argmax(cnn_model(image_test), axis=1)]
predicted_labels_untrained = cifar_class_array[tf.argmax(cnn_model_untrained(image_test), axis=1)]

model_accuracy = np.sum(label_test == predicted_labels)/len(label_test)
model_accuracy_untrained = np.sum(label_test == predicted_labels_untrained)/len(label_test)
print(f"  trained model's accuracy = {model_accuracy:0.4f}")
print(f"untrained model's accuracy = {model_accuracy_untrained:0.4f}")

  trained model's accuracy = 0.7670
untrained model's accuracy = 0.5000
